In [ ]:
import pandas as pd
import json
import requests

from io import BytesIO
from PIL import Image as PImage, ImageDraw as PImageDraw

from huggingface_hub import hf_hub_download
from ultralytics import YOLO

In [ ]:
portrait_df = pd.read_csv("https://raw.githubusercontent.com/nmolnar-parsons/major-studio-1/refs/heads/main/Project_2/Data/cleaned.csv")
portrait_df

In [ ]:
yolo_model_path = hf_hub_download(repo_id="AdamCodd/YOLOv11n-face-detection", filename="model.pt")
face_detector = YOLO(yolo_model_path)

In [ ]:
# create empty faces column
portrait_df["faces"] = ""

# iterate through csv dataset
for idx,row in list(portrait_df.iterrows())[:10]:
  # parse mediaURLs field into a python list
  mediaURLs = json.loads(row["mediaURLs"].replace("'", '"'))

  # skip if no urls
  if len(mediaURLs) < 1:
    continue

  # dowload image
  r = requests.get(mediaURLs[0])
  oimg = PImage.open(BytesIO(r.content))

  # resize to height of 256
  iw,ih = oimg.size
  nh = 256
  nw = int(iw / ih * nh)
  img = oimg.resize((nw, nh))

  # detect faces
  output = face_detector.predict(img, verbose=False)

  # if no faces, skip
  if len(output) < 1 or len(output[0]) < 1:
    continue

  # list of face boxes
  faces_xyxy = output[0].boxes.xyxy.numpy()

  # draw rect around faces (mostly for testing/debugging)
  draw = PImageDraw.Draw(img)

  for x0,y0,x1,y1 in faces_xyxy:
    draw.rectangle(((x0,y0), (x1,y1)), outline=(200,0,0), width=2)
  display(img)

  # face boxes are scaled to our smaller sized image
  #   this unscales them back to the coordinates in the original image
  unscaled_faces_xyxy = []
  for x0,y0,x1,y1 in faces_xyxy:
    unscaled_faces_xyxy.append([
      int(x0 * iw / nw),
      int(y0 * ih / nh),
      int(x1 * iw / nw),
      int(y1 * ih / nh),
    ])

  # here the unscaled_faces_xyxy can be saved back to the dataset
  portrait_df.at[idx, "faces"] = unscaled_faces_xyxy

  # or we can use it to crop faces from images
  for x0,y0,x1,y1 in unscaled_faces_xyxy:
    face_img = oimg.crop((x0,y0,x1,y1))

    # display the face
    display(face_img)
    # or save it
    # face_img.save("some_file_name.jpg")